# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
#Load dataa from weather.csv created form first assignment
weather_file = pd.read_csv("weather.csv")
weather_file.dropna()
weather_file.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Visby,0,SE,1594929614,55,57.64,18.30,63.00,4.7
1,1,Hermanus,0,ZA,1594929564,74,-34.42,19.23,53.01,3.0
2,2,Nome,75,US,1594929739,76,64.50,-165.41,57.20,4.7
3,3,Grand Gaube,4,MU,1594929390,78,-20.01,57.66,66.99,7.0
4,4,Yicheng,92,CN,1594929741,99,31.37,119.87,66.00,7.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
gmaps.configure(api_key=g_key)

In [38]:
#Locations by Lat & Lon
locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

#fig = gmaps.figure()
fig = gmaps.figure(center = [10,20] ,zoom_level = 1)

heatlayer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
#Print heat layer
fig.add_layer(heatlayer)
plt.savefig('TempPlot.png')

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
#My preferred vacation weather

HighTemp = (weather_file['Max Temp']<85)
LowTemp = (weather_file['Max Temp']>65)
BestWind = (weather_file['Wind Speed'] <10)
Clouds = (weather_file['Cloudiness']== 0)

#Ideal Weather Data
ideal_weather = weather_file.loc[(LowTemp) & (HighTemp) & (BestWind) & (Clouds)]
ideal_weather_df=pd.DataFrame(ideal_weather)

#Sort and print values
ideal_weather_df.sort_values(by=['City'], inplace=True, ascending=False)
ideal_weather_df



,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
193,193,İdil,0,TR,1594929961,15,37.34,41.89,84.45,5.75
379,379,Zadar,0,HR,1594930180,77,44.12,15.24,69.80,1.12
223,223,Yatou,0,CN,1594929996,92,37.15,122.38,66.65,5.06
211,211,Tatarsk,0,RU,1594929982,88,55.22,75.98,66.54,2.80
525,525,Takāb,0,IR,1594930359,50,36.40,47.11,65.77,6.02
125,125,Santiago del Estero,0,AR,1594929882,36,-27.80,-64.26,68.67,4.29
231,231,San Miguel de Tucumán,0,AR,1594929922,40,-26.82,-65.22,71.60,3.36
440,440,Qaryat Sulūq,0,LY,1594930257,63,31.67,20.25,75.61,9.06
532,532,Petropavl,0,KZ,1594930367,68,54.88,69.16,67.46,5.48
430,430,Mossamedes,0,AO,1594930245,62,-15.20,12.15,67.69,1.81


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
#Add hotel to data frame & print to test
Hotel_df = pd.DataFrame(ideal_weather_df)
Hotel_df["Hotel Name"]=""
Hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
193,193,İdil,0,TR,1594929961,15,37.34,41.89,84.45,5.75,
379,379,Zadar,0,HR,1594930180,77,44.12,15.24,69.80,1.12,
223,223,Yatou,0,CN,1594929996,92,37.15,122.38,66.65,5.06,
211,211,Tatarsk,0,RU,1594929982,88,55.22,75.98,66.54,2.80,
525,525,Takāb,0,IR,1594930359,50,36.40,47.11,65.77,6.02,


In [31]:
# data from Airport exercise
#Loop through the cities to find Hotel name
params = {
    "radius": 5000,
    "types": "lodging",
   # "keyword": "hotel",
    "key": g_key
}

for index, row in Hotel_df.iterrows():
    Hlat = row["Lat"]
    Hlng = row["Lng"]
    params["location"] = f"{Hlat},{Hlng}"
    
    #Place request
    get_response = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params).json()
    
    #get results
    Results = get_response['results']
    
   
    # If data is missing will try-except to skip any that are missing a data point.
    try:
        Hotel_df.loc[index, 'Hotel Name'] = Results[0]['name']
        
    except (KeyError, IndexError):
        
    
        
    

Missing field/result... skipping.
Missing field/result... skipping.


In [33]:
#Checking params and reviewing data
params = {
    "radius": 5000,
    "types": "lodging",
   # "keyword": "hotel",
    "key": g_key
}

params["location"] = "47.50,-120.50"
get_response = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params).json()
print(get_response)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 47.4952019, 'lng': -120.4778512}, 'viewport': {'northeast': {'lat': 47.4964687302915, 'lng': -120.4763081197085}, 'southwest': {'lat': 47.4937707697085, 'lng': -120.4790060802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'f23d4fcc952061d8ec192a0a41fd155c31768f3f', 'name': 'Cashmere Cider Mill Guest Suites, Weddings & Tasting Room', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2268, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102074872369109954604">Ed Ellis</a>'], 'photo_reference': 'CmRaAAAAedRLdF5swE0XOuGyy0VwF3SWbERgbIy1Z4MxHfRhh4p4caLew2gX9uB1y0tVp4HIvVlfvv41qStHNQXCJiXLc2IxFmpz3PAO24F92EzsqKpLtr0kjhg95YXrllyWDhM4EhAQXtsCAOio53N76MqBx52xGhRETjy0LraF3lOo27DyLFEce7ILeQ', 'width': 4032}], 'place_id': 'ChIJg0RctJdKmlQRQyAQTXgJTNg', 'plus_code': {'compound_code': 'FGWC+3V Cashmere, WA, USA', '

In [34]:
#Print dataframe with hotels names
Hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
193,193,İdil,0,TR,1594929961,15,37.34,41.89,84.45,5.75,ÇAKIR APARTMANI
379,379,Zadar,0,HR,1594930180,77,44.12,15.24,69.80,1.12,Hotel Kolovare
223,223,Yatou,0,CN,1594929996,92,37.15,122.38,66.65,5.06,GreenTree Hotel Rongcheng Bus Station
211,211,Tatarsk,0,RU,1594929982,88,55.22,75.98,66.54,2.80,Gostinitsa Neptun Tatarsk
525,525,Takāb,0,IR,1594930359,50,36.40,47.11,65.77,6.02,مهمانپذیر تخت سلیمان


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map

markerlayer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markerlayer)

#Print - display fig
fig





Figure(layout=FigureLayout(height='420px'))